In [1]:
import torch
from torch.utils.tensorboard import SummaryWriter
torch.__version__

'1.7.1'

In [2]:
available_device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{available_device} is available")
device = torch.device(device=available_device)
device

cuda is available


device(type='cuda')

## Preparing Dataset

In [3]:
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
batch_size = 64

In [4]:
train_set = datasets.CIFAR10(root="pytorch_dataset",
                           download=True,
                           transform=transforms.ToTensor()
                          )
test_set = datasets.CIFAR10(root="pytorch_dataset",
                           train=False,
                           download=True,
                           transform=transforms.ToTensor()
                          )
print("done")

Files already downloaded and verified
Files already downloaded and verified
done


In [5]:
train_loader = DataLoader(dataset=train_set, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(dataset=test_set, shuffle=True, batch_size=batch_size)

## Creating model

In [8]:
import torch.nn.functional as F
import torch.nn as nn

In [9]:
class CNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(16*5*5,128)
        self.fc2 = nn.Linear(128,num_classes)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.reshape(x.shape[0], -1) # flatten CONV -> FC
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x    

## model initialized


In [11]:
n_channels = 3
num_classes = 10
lr = 0.001

In [12]:
### cnn model initialized using the CNN class defined earlier
cnn_model = CNN(n_channels, num_classes).to(device=device)

In [14]:
criterion = nn.CrossEntropyLoss()

### optimizer


In [13]:
from torch.optim import Adam
optimizer = Adam(cnn_model.parameters(), lr=lr)

## TensorBoard

In [19]:
writer = SummaryWriter(f'runs/CIFAR10/tryingout_tensorboard')

## Training the model

In [16]:
# losses = []
# accuracies = []

In [23]:
num_epochs = 3
batch_sizes = [64]
learning_rates = [0.01]

In [22]:
for batch_size in batch_size:
    for lr in learning_rates:
        step = 0
        for epoch in range(num_epochs):
            losses = []
            accuracies = []

            for x_train, y_train in train_loader:
                x_train = x_train.to(device)
                y_train = y_train.to(device)
        #         x_train = x_train/255

                # zero the parameter gradients
                optimizer.zero_grad()

                y_pred = cnn_model(x_train)

                loss = criterion(y_pred, y_train)
                losses.append(loss.item())
                loss.backward()

                optimizer.step()

                _, predictions = y_pred.max(1)

                num_correct = (predictions==y_train).sum()
                running_train_acc = float(num_correct)/float(x_train.shape[0])

                writer.add_scalar('Training Loss', loss, global_step=step)
                writer.add_scalar("Training Accuracy",running_train_acc, global_step=step)
                step+=1
            print(f" loss at the end of {epoch} epoch is {loss}")

print("Done Training")

 loss at the end of 0 epoch is 1.247835636138916
 loss at the end of 1 epoch is 0.9533640146255493
 loss at the end of 2 epoch is 1.0991328954696655
Done Training
